# Analiza morfometryczna zlewni

Morfometria zlewni to analiza cech fizycznych i geometrycznych zlewni. Hydrolog oferuje trzy moduły:

1. **WatershedGeometry** - wskaźniki kształtu zlewni
2. **TerrainAnalysis** - parametry wysokościowe i spadki
3. **HypsometricCurve** - krzywa hipsograficzna

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

from hydrolog.morphometry import WatershedGeometry, TerrainAnalysis, HypsometricCurve
import numpy as np

## 1. Wskaźniki kształtu zlewni

Kształt zlewni wpływa na kształt hydrogramu odpływu. Zlewnia wydłużona generuje hydrogram bardziej spłaszczony niż zlewnia zwarta.

### Parametry wejściowe
- Powierzchnia A [km²]
- Obwód P [km]
- Długość zlewni L [km] - odległość od ujścia do najdalszego punktu

In [ ]:
# Parametry przykładowej zlewni
area_km2 = 45.0
perimeter_km = 32.0
length_km = 12.0

# Utwórz obiekt geometrii
geom = WatershedGeometry(
    area_km2=area_km2,
    perimeter_km=perimeter_km,
    length_km=length_km
)

print("PARAMETRY GEOMETRYCZNE")
print("=" * 40)
params = geom.get_parameters()
print(f"Powierzchnia: {params.area_km2} km²")
print(f"Obwód: {params.perimeter_km} km")
print(f"Długość: {params.length_km} km")
print(f"Szerokość średnia: {params.width_km:.2f} km")

### Wskaźniki kształtu

| Wskaźnik | Wzór | Interpretacja |
|----------|------|---------------|
| Wskaźnik formy (Horton) | Cf = A/L² | bliski 1 = zwarta |
| Wskaźnik zwartości (Gravelius) | Cz = P/(2√πA) | 1 = idealne koło |
| Wskaźnik kolistości (Miller) | Ck = 4πA/P² | 1 = idealne koło |
| Wskaźnik wydłużenia (Schumm) | Ce = (2/L)√(A/π) | 1 = koło |
| Wskaźnik lemniskatyczny | Cl = L²/(4A) | kształt lemniskaty |

In [ ]:
# Oblicz wskaźniki kształtu
indicators = geom.get_shape_indicators()

print("\nWSKAŹNIKI KSZTAŁTU")
print("=" * 50)
print(f"Wskaźnik formy (Cf):        {indicators.form_factor:.3f}")
print(f"Wskaźnik zwartości (Cz):    {indicators.compactness_coefficient:.3f}")
print(f"Wskaźnik kolistości (Ck):   {indicators.circularity_ratio:.3f}")
print(f"Wskaźnik wydłużenia (Ce):   {indicators.elongation_ratio:.3f}")
print(f"Wskaźnik lemniskatyczny:    {indicators.lemniscate_ratio:.3f}")

In [ ]:
# Interpretacja kształtu
def interpret_shape(indicators):
    """Interpretacja wskaźników kształtu."""
    print("\nINTERPRETACJA")
    print("-" * 50)
    
    # Wskaźnik formy
    if indicators.form_factor > 0.5:
        print("→ Zlewnia zwarta (Cf > 0.5)")
    elif indicators.form_factor > 0.3:
        print("→ Zlewnia pośrednia (Cf = 0.3-0.5)")
    else:
        print("→ Zlewnia wydłużona (Cf < 0.3)")
    
    # Wskaźnik zwartości
    if indicators.compactness_coefficient < 1.25:
        print("→ Kształt zbliżony do koła (Cz < 1.25)")
    elif indicators.compactness_coefficient < 1.50:
        print("→ Kształt owalny (Cz = 1.25-1.50)")
    else:
        print("→ Kształt wydłużony (Cz > 1.50)")
    
    # Wpływ na odpływ
    print("\nWPŁYW NA ODPŁYW:")
    if indicators.form_factor > 0.4:
        print("→ Wysoki, ostry szczyt hydrogramu")
        print("→ Krótki czas odpływu")
    else:
        print("→ Niższy, bardziej rozciągnięty hydrogram")
        print("→ Dłuższy czas odpływu")

interpret_shape(indicators)

## 2. Parametry terenu

Analiza wysokościowa i spadków zlewni.

In [ ]:
# Parametry terenu
terrain = TerrainAnalysis(
    elevation_min_m=150.0,    # Minimalna wysokość (ujście)
    elevation_max_m=520.0,    # Maksymalna wysokość
    length_km=12.0,           # Długość zlewni
    elevation_mean_m=340.0,   # Średnia wysokość (opcjonalna)
    channel_length_km=10.5    # Długość cieku głównego (opcjonalna)
)

# Parametry wysokościowe
elev = terrain.get_elevation_parameters()
print("PARAMETRY WYSOKOŚCIOWE")
print("=" * 40)
print(f"Wysokość minimalna: {elev.elevation_min_m} m n.p.m.")
print(f"Wysokość maksymalna: {elev.elevation_max_m} m n.p.m.")
print(f"Wysokość średnia: {elev.elevation_mean_m} m n.p.m.")
print(f"Deniwelacja: {elev.relief_m} m")

In [ ]:
# Spadki
slope = terrain.get_slope_parameters()
print("\nPARAMETRY SPADKU")
print("=" * 40)
print(f"Spadek zlewni: {slope.watershed_slope_m_per_m:.4f} m/m ({slope.watershed_slope_percent:.2f}%)")
if slope.channel_slope_m_per_m is not None:
    print(f"Spadek cieku: {slope.channel_slope_m_per_m:.4f} m/m ({slope.channel_slope_percent:.2f}%)")

## 3. Krzywa hipsograficzna

Krzywa hipsograficzna pokazuje rozkład powierzchni zlewni w zależności od wysokości. Jest podstawą do określenia stadium erozji zlewni.

### Całka hipsograficzna (HI)
- HI > 0.6 - młode stadium (mała erozja)
- HI = 0.4-0.6 - dojrzałe stadium
- HI < 0.4 - stare stadium (silna erozja)

In [ ]:
# Symulacja danych DEM (w praktyce z pliku rastrowego)
np.random.seed(42)

# Generujemy rozkład wysokości typowy dla zlewni dojrzałej
# (więcej powierzchni w średnich wysokościach)
n_cells = 10000
elevation_min = 150.0
elevation_max = 520.0

# Rozkład beta dający więcej komórek w średnich wysokościach
beta_vals = np.random.beta(2, 2, n_cells)
elevations = elevation_min + beta_vals * (elevation_max - elevation_min)

print(f"Wygenerowano {n_cells} komórek DEM")
print(f"Zakres wysokości: {elevations.min():.1f} - {elevations.max():.1f} m")

In [ ]:
# Analiza hipsograficzna
hypso = HypsometricCurve(elevations)
result = hypso.analyze(n_points=101)

print("ANALIZA HIPSOGRAFICZNA")
print("=" * 50)
print(f"Wysokość minimalna: {hypso.elevation_min:.1f} m")
print(f"Wysokość maksymalna: {hypso.elevation_max:.1f} m")
print(f"Deniwelacja: {hypso.relief:.1f} m")
print(f"\nWysokość średnia: {result.elevation_mean_m:.1f} m")
print(f"Wysokość mediana: {result.elevation_median_m:.1f} m")
print(f"\nCałka hipsograficzna: {result.hypsometric_integral:.3f}")

In [ ]:
# Interpretacja całki hipsograficznej
hi = result.hypsometric_integral

print("\nINTERPRETACJA STADIUM EROZJI")
print("-" * 50)
if hi > 0.6:
    print(f"HI = {hi:.3f} → Stadium młode (youthful)")
    print("→ Mała erozja, krzywa wypukła")
    print("→ Dominują procesy denudacji")
elif hi > 0.4:
    print(f"HI = {hi:.3f} → Stadium dojrzałe (mature)")
    print("→ Umiarkowana erozja, krzywa S-kształtna")
    print("→ Równowaga między erozją a akumulacją")
else:
    print(f"HI = {hi:.3f} → Stadium stare (old)")
    print("→ Zaawansowana erozja, krzywa wklęsła")
    print("→ Dominują procesy akumulacji")

In [ ]:
# Percentyle wysokości
print("\nPERCENTYLE WYSOKOŚCI")
print("-" * 40)
for p in [10, 25, 50, 75, 90]:
    elev = hypso.elevation_at_percentile(p)
    print(f"{p}% powierzchni powyżej: {elev:.1f} m")

## 4. Wizualizacja

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Histogram wysokości
    axes[0].hist(elevations, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].axvline(result.elevation_mean_m, color='red', linestyle='--', 
                    label=f'Średnia: {result.elevation_mean_m:.0f} m')
    axes[0].axvline(result.elevation_median_m, color='green', linestyle='--',
                    label=f'Mediana: {result.elevation_median_m:.0f} m')
    axes[0].set_xlabel('Wysokość [m n.p.m.]')
    axes[0].set_ylabel('Liczba komórek')
    axes[0].set_title('Rozkład wysokości')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Krzywa hipsograficzna (względna)
    axes[1].plot(result.relative_areas, result.relative_heights, 'b-', linewidth=2)
    axes[1].fill_betweenx(result.relative_heights, result.relative_areas, alpha=0.3)
    axes[1].set_xlabel('Względna powierzchnia a/A [-]')
    axes[1].set_ylabel('Względna wysokość h/H [-]')
    axes[1].set_title(f'Krzywa hipsograficzna (HI = {result.hypsometric_integral:.3f})')
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)
    axes[1].grid(True, alpha=0.3)
    
    # Krzywa hipsograficzna (bezwzględna)
    abs_heights = hypso.elevation_min + result.relative_heights * hypso.relief
    axes[2].plot(result.relative_areas * 100, abs_heights, 'b-', linewidth=2)
    axes[2].fill_betweenx(abs_heights, result.relative_areas * 100, alpha=0.3)
    axes[2].set_xlabel('Powierzchnia powyżej [%]')
    axes[2].set_ylabel('Wysokość [m n.p.m.]')
    axes[2].set_title('Krzywa hipsograficzna (bezwzględna)')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib nie jest zainstalowany.")

## 5. Porównanie krzywych hipsograficznych różnych stadiów

In [ ]:
# Generujemy trzy typy zlewni
np.random.seed(42)

# Młoda (wypukła) - więcej wysokich terenów
elev_young = elevation_min + np.random.beta(3, 1.5, n_cells) * (elevation_max - elevation_min)

# Dojrzała (S-kształtna)
elev_mature = elevation_min + np.random.beta(2, 2, n_cells) * (elevation_max - elevation_min)

# Stara (wklęsła) - więcej niskich terenów
elev_old = elevation_min + np.random.beta(1.5, 3, n_cells) * (elevation_max - elevation_min)

# Analizy
hypso_young = HypsometricCurve(elev_young)
hypso_mature = HypsometricCurve(elev_mature)
hypso_old = HypsometricCurve(elev_old)

result_young = hypso_young.analyze()
result_mature = hypso_mature.analyze()
result_old = hypso_old.analyze()

print("PORÓWNANIE STADIÓW EROZJI")
print("=" * 50)
print(f"{'Stadium':<15} {'HI':>10} {'H_mean [m]':>12} {'H_med [m]':>12}")
print("-" * 50)
for name, res in [('Młode', result_young), ('Dojrzałe', result_mature), ('Stare', result_old)]:
    print(f"{name:<15} {res.hypsometric_integral:>10.3f} {res.elevation_mean_m:>12.1f} {res.elevation_median_m:>12.1f}")

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Krzywe hipsograficzne
    for res, color, label in [
        (result_young, 'green', f'Młoda (HI={result_young.hypsometric_integral:.2f})'),
        (result_mature, 'blue', f'Dojrzała (HI={result_mature.hypsometric_integral:.2f})'),
        (result_old, 'red', f'Stara (HI={result_old.hypsometric_integral:.2f})')
    ]:
        ax.plot(res.relative_areas, res.relative_heights, color=color, 
                linewidth=2, label=label)
    
    # Linia odniesienia (idealne koło)
    ax.plot([0, 1], [1, 0], 'k--', alpha=0.3, label='Linia odniesienia')
    
    ax.set_xlabel('Względna powierzchnia a/A [-]', fontsize=12)
    ax.set_ylabel('Względna wysokość h/H [-]', fontsize=12)
    ax.set_title('Porównanie krzywych hipsograficznych', fontsize=14)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.legend(loc='lower left')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib nie jest zainstalowany.")

## Podsumowanie

### Moduły morfometrii Hydrolog:

| Moduł | Dane wejściowe | Wyniki |
|-------|---------------|--------|
| WatershedGeometry | A, P, L | Wskaźniki kształtu |
| TerrainAnalysis | H_min, H_max, L | Spadki, deniwelacja |
| HypsometricCurve | DEM (wysokości) | Krzywa, HI, percentyle |

### Zastosowania:
- Charakterystyka zlewni do modelowania
- Określenie stadium erozji
- Porównywanie zlewni
- Parametry wejściowe do wzorów hydrologicznych